# 영화 추천

## 영화 년도별 데이터 생성 및 저장

In [1]:
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
def replace_content(content):
    content = content.replace('\n', '')
    content = content.replace('\t', '')
    content = content.replace('\xa0', '')
    return content

In [ ]:
def get_link(year: int, pages: int):
    links = []
    for page in range(pages):
        url = f"https://movie.naver.com/movie/sdb/browsing/bmovie.naver?open={year}&page={page+1}"
        req = requests.get(url) 
        soup = BeautifulSoup(req.text, 'lxml')
        movielinks = soup.select('ul.directory_list > li > a[href] ') 
        for movielink in movielinks:
            link = "https://movie.naver.com"+str(movielink.get('href'))
            if link not in links:
                links.append(link)
    return links

In [ ]:
# 년도별로 데이터를 저장하기 위해서 년도와 페이지 입력으로 크롤링

links = get_link(2021, 3)

In [ ]:
movie_id = []
titles = []
contents = []
ranks = []
parts = []

In [ ]:
url = "https://www.naver.com"
browser = wd.Chrome("../chromedriver_win32/chromedriver.exe") 
browser.get(url) 
time.sleep(3.0) # 30 
browser.find_element_by_css_selector('body').send_keys(Keys.CONTROL + "t") 

for link in links: 
    browser.switch_to.window(browser.window_handles[-1]) 
    time.sleep(0.1) 
    browser.get(link) 
    time.sleep(0.1) 
    browser.switch_to.window(browser.window_handles[0]) 
    time.sleep(0.3) 
    html_soup = BeautifulSoup(browser.page_source, 'lxml') 
    flag = html_soup.text[0:10] 
    newflag = "".join(flag) 
    newflag = newflag.replace('\n', '')
    if newflag != '':
        soup = BeautifulSoup(browser.page_source, 'lxml')
        
        rank = soup.find('a', id='pointNetizenPersentBasic')
        content = soup.find('p', class_='con_tx')
        if rank == None or content == None:
            continue
        ranks.append(rank.get_text().replace('\n', ''))

        content = content.get_text()
        contents.append(replace_content(content))
        
        title = soup.find('h3', class_='h_movie')
        titles.append(title.a.get_text())
        
        movie_id.append(title.a['href'][-6:].replace('=', ''))

        part = soup.select_one('dd > p > span > a')
        parts.append(part.get_text())

In [ ]:
print(movie_id)

In [ ]:
def save_to_data(num: int, vmoie_id: list, title: list, part: list, rank: list, content: list):
    try:
        df = pd.DataFrame({'영화id':movie_id, '제목':title, '장르':part, '평점':rank, '줄거리':content})
        df.to_csv(f'data_{num}.csv', index=False)
    except:
        print('error')

In [ ]:
# 년도별로 저장

save_to_data(1940, movie_id, titles, parts, ranks, contents)

In [ ]:
pd.read_csv('data_1940.csv')

In [ ]:
df=pd.read_csv('data_2021.csv')
for i in range(2020, 1989, -1):
    df=df.append(pd.read_csv('data_{}.csv'.format(i)))
for i in range(1980, 1930, -10):
    df=df.append(pd.read_csv('data_{}.csv'.format(i)))

In [ ]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8-sig')